In [241]:
import random 
import math 
random.seed(42)
array = [random.randint(0,1) for _ in range(1_00_000)]
#array = ''.join([str(x) for x in array])

In [226]:
def prefixSum(arr):
    n = len(arr)
    rank = [0]*n
    for i in range(n):
        rank[i] = rank[i-1] + int(arr[i])
    return rank

In [3]:
ranked_array = prefixSum(array)

In [308]:
x = 'abcd'
x[1:2]

'b'

In [ ]:
first = []
second = []
l = math.ceil(math.log2(len(array))/2)**2
k = math.ceil(math.log2(len(array))/2)

for i in range(0,len(array),l):
    first.append(ranked_array[i])

for j in range(0,len(array),k):
    i_l,j_l = divmod(j,l)
    if j_l != 0:        
        second.append(sum(array[i_l*l+1:j+1]))
    else:
        second.append(0)

from itertools import product
third = {}
for r in list(product(['0','1'],repeat=k-1)):
   third[''.join(r)] =  prefixSum([int(x) for x in r])

def rank(first,second,third,B,i):
    n = len(B)
    l = math.ceil(math.log2(n)/2)**2
    k = math.ceil(math.log2(n)/2)
    i_l,j_l = divmod(i,l)
    i_k,j_k = divmod(i,k)
    rank_l = first[i_l]
    rank_l += second[i_k]
    if j_k > 0:
        c = B[((k*i_k)+1):k*(i_k+1)]
        if len(c) < k - 1:
            c += '0'*(k -1 - len(c))
        rank_l += third[c][j_k - 1]

    return rank_l

In [ ]:
def createBlocks(array):
    first = []
    second = []
    l = math.ceil(math.log2(len(array))/2)**2
    k = math.ceil(math.log2(len(array))/2)
    ranked_array = prefixSum(array)
    for i in range(0,len(array),l):
        first.append(ranked_array[i])

    for j in range(0,len(array),k):
        i_l,j_l = divmod(j,l)
        if j_l != 0:        
            second.append(ranked_array[j] - ranked_array[i_l*l])
        else:
            second.append(0)

    from itertools import product
    third = {}
    for r in list(product(['0','1'],repeat=k-1)):
        third[''.join(r)] =  prefixSum([int(x) for x in r])
        
    return first,second,third,array

In [54]:
[rank(first,second,third,array,i) for i in range(1_00_000)] == ranked_array

True

In [342]:
query = ''
with open('Homo_sapiens.GRCh38.dna.chromosome.22.fa') as file:
    for line in file:
        if line.startswith('>'):
            continue
        else:
            query += line.strip()


KeyboardInterrupt: 

In [343]:
## wavelet tree 
strValue = {'A':1,'C':2,'G':3,'T':4,'N':5}

class Node:
    def __init__(self,key=None,value=None):
        self.key = key
        self.value = value 
        self.left = None
        self.right = None
        self.first = None 
        self.second = None 
        self.third = None
        self.bitArray = None

nodes = []
for key,value in strValue.items():
    nodes.append(Node(value,key))

while len(nodes) > 1: 
    new_level = []
    for i in range(0,len(nodes),2):
        if i+1 < len(nodes):
            left,right = nodes[i],nodes[i+1]
            parent = Node()
            parent.left = left 
            parent.right = right
            temp = left
            while temp.right is not None:
                temp = temp.right

            parent.key = temp.key 
            new_level.append(parent)
        else:
            new_level.append(nodes[i])
    nodes = new_level

root = nodes[0]

In [339]:
def bitstring(tree,string): 
    if tree.left is not None and tree.right is not None: 
        string0 = ''
        for key,value in strValue.items():
            if value <= tree.key:
                string0 += key
        bitarray = ''.join(['0' if s in string0 else '1' for s in string])
        tree.first, tree.second, tree.third, tree.bitArray = createBlocks(bitarray)
        leftSubstring = ''
        rightSubstring = ''
        for i,b in enumerate(bitarray):
            if b == '0':
                leftSubstring += string[i]
            else:
                rightSubstring += string[i]
        if len(leftSubstring) > 0: 
            bitstring(tree.left,leftSubstring)
        if len(rightSubstring) > 0: 
            bitstring(tree.right,rightSubstring)

In [344]:
bitstring(root,query)

In [334]:
def rankQuery(tree,alphabet,i): 
    while tree.left is not None and tree.right is not None and tree.bitArray is not None:
        ps = rank(tree.first,tree.second,tree.third,tree.bitArray,i-1)
        if strValue[alphabet] <= tree.key: 
            i = i  - ps 
            tree = tree.left 
        else: 
            tree = tree.right
            i = ps 
    return i 

In [335]:
from collections import Counter

In [345]:
import time 
start_time = time.time()
res = Counter(query)
end_time = time.time()

print(f'Counter Time = {end_time - start_time}')
start_time = time.time()
res_dict = {}
for i in ['A','T','G','C','N']:
    res_dict[i] = rankQuery(root,i,len(query))
end_time = time.time() 
print(f'rankQuery time = {end_time - start_time}')

Counter Time = 0.6324152946472168
rankQuery time = 0.0


In [346]:
res

Counter({'N': 11321733, 'T': 996840, 'A': 982185, 'G': 672719, 'C': 608443})

In [347]:
res_dict

{'A': 982185, 'T': 996840, 'G': 672719, 'C': 608443, 'N': 11321733}

In [214]:
from pympler import asizeof

In [348]:
asizeof.asizesof(root)

(44413880,)

In [349]:
asizeof.asizesof(query)

(14581968,)